##### Тестирование показателя max_leas_ovd_pay_dt - дата последней оплаченной просроченной задолженности по лизинговым платежам с ЦАЛ

сравнение  "data/external/last_overdue_dt_tsal_for_test.pkl" - дата последней оплаченной просроченной задолженности по лизинговым платежам по договорам ЦАЛ с "data/interm/test/leas_ovd_pay_dt.pkl" - рассчитанные даты последней оплаченной задолженности

In [12]:
pre = '../../' # путь до главной директории проекта
%run ../../params.ipynb #параметры как логин пароль к БИ, параметры расчёта ( в нашем случае их нет)
%run ../init.ipynb #подгрузить нужные библиотеки

In [13]:
# сюда таблицы служебные будем кидать, для отчетов
name = pre + "data/interm/auxilary/test_dsnc_ovd_pay.xlsx"
writer = ExcelWriter(name)

In [14]:
name = pre + "data/external/last_overdue_dt_tsal_for_test.pkl"
last_overdue_dt_tsal = pd.read_pickle(name, compression={'method':'zip'})

print(len(last_overdue_dt_tsal))

num, duplicates = identify_duplicates(last_overdue_dt_tsal, 'CONTRACT_KEY')
Check_dupl(num, duplicates)

last_overdue_dt_tsal['CONTRACT_KEY'] = last_overdue_dt_tsal['CONTRACT_KEY'].astype(str)
# last_overdue_dt_tsal.head(1)

167757


In [15]:
name = pre + "data/interm/test/leas_ovd_pay_dt.pkl"
test_leas_ovd_pay = pd.read_pickle(name, compression={'method':'zip'})

print(len(test_leas_ovd_pay))
# test_leas_ovd_pay.head(1)

108315


In [16]:
%run ../init.ipynb #подгрузить нужные библиотеки

kwargs = dict( sourcesName = ['tsal', 'target'], 
               suffix = '_target',
               on = 'CONTRACT_KEY',  
               notnull_col = ['SNAPSHOT_DT', 'decision_date'],
               base_col = ['CONTRACT_NUM', 'CONTRACT_KEY', 'INN', 'decision_date']
             )

comp =  ComparisonDataFrames(last_overdue_dt_tsal, test_leas_ovd_pay, **kwargs)   

In [17]:
missed = comp.missedRows(sourcesName='tsal')

отсутствуют в tsal 6 строк


In [18]:
df = comp.presentRows()
# df.head(1)

одновременно присутствуют в tsal и target 108309 строк


полностью совпасть не сможем, так как платежи у нас после перерасчетов

In [19]:
df = comp.check_fields('MAX_LEAS_OVD_PAY_DT', 'MAX_LEAS_OVD_PAY_DT')
# df.head(1)

значения совпадают во всех 108309 строках


1с подтверждает данные посчитанные здесь а не витрине ЦАЛ

In [21]:
l = len(test_leas_ovd_pay[test_leas_ovd_pay['max_leas_ovd_pay_dt'] > test_leas_ovd_pay['decision_date']])
assert l == 0